# Urejanje besedilnih podatkov in regularni izrazi

Kot smo povedali v prejšnjih predavanjih, je spletna enciklopedija [Wikipedia](https://www.wikipedia.org/) bogat vir podatkovnih tabel, kot je na primer [tabela s podatki o bruto domačem proizvodu držav](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)).

V nadaljevanju bomo spoznali funkcije za uvoz podatkov iz spletnih strani. Pri tem uvozu imamo pogosto opravka z besedili, ki jih je treba pri nadaljnjem urejanju podatkov obdelovati in prečistiti. Priročno orodje za učinkovito obdelavo besedilnih podatkov in nizov znakov so regularni izrazi, ki jih bomo spoznali v drugem delu predavanja.

## Uvoz podatkov s spleta, knjižnici `requests` in `bs4`

Za uvoz spletnih strani uporabimo lahko Python-ovsko knjižnico `request`. Funkcija `get` v tej knjižnici nam omogoča enostaven uvoz: njen prvi argument je [URL](https://en.wikipedia.org/wiki/URL) ali, poenostavljeno povedano, naslov spletne strani. Preberimo [članek o Univerzi v Ljubljani](https://sl.wikipedia.org/wiki/Univerza_v_Ljubljani), ki ga ponuja slovenska različica spletne enciklopedije Wikipedia:

In [1]:
import requests

html = requests.get("https://sl.wikipedia.org/wiki/Univerza_v_Ljubljani")
print(html)

<Response [200]>


Rezultat uvoza je objekt razreda `Response`, ki ima dva ključna atributa `status_code` in `content`:

In [2]:
print(html.status_code)
print(html.content)

200
b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" lang="sl" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Univerza v Ljubljani - Wikipedija, prosta enciklopedija</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disa

Vrednost atributa `status_code` nam pove ali je bil uvoz spletne strani uspešen. Koda `200` pomeni, da je uvoz bil uspešno opravljen. Kode, ki se začenjajo z `4` nam sporočajo, da je prišlo do napake. Pogosta napaka je `404`, kar pomeni, da URL ne obstaja, t.j., da spletna stran s podanim naslovom (URL) ne obstaja. V članku [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) spletne enciklopedije Wikipedia dobite seznam možnih vrednosti kode in njihovih pomenov.

Vrednost atributa `content` je niz znakov, ki vsebuje izvorno kodo spletne strani v označevalnem jeziku [HTML](https://en.wikipedia.org/wiki/HTML). Slednjega ste spoznali v okviru predmeta Računalniški praktikum v prvem letniku.

Izvorna koda izbrane spletne strani vsebuje naštevni seznam fakultet, članic Univerze v Ljubljani. Naš cilj je dobiti seznam imen fakultet in ga spraviti v običajno zaporedje `pandas`. Zato, da pridobimo posamezne elemente izvorne kode spletne strani (npr. naštevne sezname, razdelke ali povezave) rabimo razčlenjevalnik (angl. _parser_) za označevalni jezik HTML. Python-ovska knjižnica `bs4` in njena osnovna funkcija `BeautifulSoup` implementira ravno tak razčlenjevalnik za označevalni jezik HTML:

In [3]:
from bs4 import BeautifulSoup

juha = BeautifulSoup(html.content, 'html.parser')
print(type(juha))
print(juha)

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="sl">
<head>
<meta charset="utf-8"/>
<title>Univerza v Ljubljani - Wikipedija, prosta enciklopedija</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-ma

Rezultat funkcije je objekt razreda `BeautifulSoup`, ki ponuja veliko število uporabnih metod za dostop do posameznih elementov spletne strani. Zalo priročna in pogosto uporabljena metoda `select` nam omogoča izbiro elementov spletne strani z uporabo izbirnikov CSS (angl. _CSS selectors_), glej ustrezen razdelek v članku [CSS](https://en.wikipedia.org/wiki/CSS).

Na tem mestu seveda moramo sestaviti ustrezen izbirnik CSS, ki bo iz članka o Univerzi v Ljubljani izbral tisti element, ki vsebuje seznam fakultet. Iz ogleda spletne strani je jasno, da gre za naštevni seznam v razdelku z naslovom _Trenutne članice_. Za izbirnik CSS bomo morali pogledati izvorno kodo tega dela spletne strani. V različnih spletnih brskalnikih to dosežemo na različne načine, a večinoma z uporabo menijske postavke `Inspect`. V brskalniku `Firefox` desni klik na naslov razdelka nam odpre meni s postavko `Inspect`. Slednja nam odpre izvorno kodo spletne strani ravno na tem mestu, ki se nanaša na izbran razdelek. Poglejmo izpis

>		<h4>
>		<span id="Trenutne_.C4.8Dlanice"></span>
>		<span class="mw-headline" id="Trenutne_članice">Trenutne članice</span>
>		<span class="mw-editsection">...</span>
>		</h4>
>
>		<div class="{{references-small" style="-webkit-column-count:3; -moz-column-count:3; column-count:3};">
>		<ul>...</ul>
>		</div>

V predzadnji vrstici zgornje kode `<ul>...</ul>` je naštevni seznam z imeni fakultet Univerze v Ljubljani, ki nas zanima. Izberemo ga posredno, skozi prejšnji element v izvorni kodi, to je naslov razdelka, ki ga definira element z oznako `h4`. Ker je takih elementov v spletni strani običajno več, bomo določili bolj specifičen izbirnik `h4:contains('Trenutne članice')` zato, da izberemo točno tisti razdelek, ki nas zanima, in ne kateregakoli razdelka z oznako `h4`. Ker nas zanima **naslednji** element v izvorni kodi z oznako `div`, izbirniku dodamo znak `+` (naslednji element) in izbrano oznako (v tem primeru `div`).

Torej, sestavili smo izbirnik CSS `h4:contains('Trenutne članice')+ div`, ki ga uporabimo v klicu metode `select`:

In [4]:
nastevni_seznam_html = juha.select("h4:contains('Trenutne članice')+ div")
print(nastevni_seznam_html)

[<div class="{{references-small" style="-webkit-column-count:3; -moz-column-count:3; column-count:3};">
<ul><li><a href="/wiki/Akademija_za_glasbo_v_Ljubljani" title="Akademija za glasbo v Ljubljani">Akademija za glasbo</a></li>
<li><a href="/wiki/Akademija_za_gledali%C5%A1%C4%8De,_radio,_film_in_televizijo_v_Ljubljani" title="Akademija za gledališče, radio, film in televizijo v Ljubljani">Akademija za gledališče, radio, film in televizijo</a></li>
<li><a href="/wiki/Akademija_za_likovno_umetnost_in_oblikovanje_v_Ljubljani" title="Akademija za likovno umetnost in oblikovanje v Ljubljani">Akademija za likovno umetnost in oblikovanje</a></li>
<li><a href="/wiki/Biotehni%C5%A1ka_fakulteta_v_Ljubljani" title="Biotehniška fakulteta v Ljubljani">Biotehniška fakulteta</a></li>
<li><a href="/wiki/Ekonomska_fakulteta_v_Ljubljani" title="Ekonomska fakulteta v Ljubljani">Ekonomska fakulteta</a></li>
<li><a href="/wiki/Fakulteta_za_arhitekturo_v_Ljubljani" title="Fakulteta za arhitekturo v Ljublja

/Users/ljupco/predmeti/pandas/venv/lib/python3.11/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Rezultat funkcije `select` je seznam vseh elementov HTML iz izvorne kode spletne strani, ki ustrezajo podanemu izbiriku CSS. V tem primeru, ko smo podali zelo specifičen izbirnik, ima seznam zgolj en element:

In [5]:
nastevni_seznam_fakultet = nastevni_seznam_html[0]
print(nastevni_seznam_fakultet)

<div class="{{references-small" style="-webkit-column-count:3; -moz-column-count:3; column-count:3};">
<ul><li><a href="/wiki/Akademija_za_glasbo_v_Ljubljani" title="Akademija za glasbo v Ljubljani">Akademija za glasbo</a></li>
<li><a href="/wiki/Akademija_za_gledali%C5%A1%C4%8De,_radio,_film_in_televizijo_v_Ljubljani" title="Akademija za gledališče, radio, film in televizijo v Ljubljani">Akademija za gledališče, radio, film in televizijo</a></li>
<li><a href="/wiki/Akademija_za_likovno_umetnost_in_oblikovanje_v_Ljubljani" title="Akademija za likovno umetnost in oblikovanje v Ljubljani">Akademija za likovno umetnost in oblikovanje</a></li>
<li><a href="/wiki/Biotehni%C5%A1ka_fakulteta_v_Ljubljani" title="Biotehniška fakulteta v Ljubljani">Biotehniška fakulteta</a></li>
<li><a href="/wiki/Ekonomska_fakulteta_v_Ljubljani" title="Ekonomska fakulteta v Ljubljani">Ekonomska fakulteta</a></li>
<li><a href="/wiki/Fakulteta_za_arhitekturo_v_Ljubljani" title="Fakulteta za arhitekturo v Ljubljan

To je objekt razreda `element.Tag`, ki ga knjižnica `b4` uporablja za spravljanje elementov HTML. Iz osnov označevalnega jezika HTML se spomnimo, da posamezne postavke neurejenega naštevnega seznama z oznako `ul` označimo z oznako `li`. Metoda razreda ``element.Tag`, `find_all` nam omogoča luščenje seznama elementov spletne strani, ki imajo izbrano oznako (v tem primeru `li`):

In [6]:
fakultete_html = nastevni_seznam_fakultet.find_all("li")
print(fakultete_html)

[<li><a href="/wiki/Akademija_za_glasbo_v_Ljubljani" title="Akademija za glasbo v Ljubljani">Akademija za glasbo</a></li>, <li><a href="/wiki/Akademija_za_gledali%C5%A1%C4%8De,_radio,_film_in_televizijo_v_Ljubljani" title="Akademija za gledališče, radio, film in televizijo v Ljubljani">Akademija za gledališče, radio, film in televizijo</a></li>, <li><a href="/wiki/Akademija_za_likovno_umetnost_in_oblikovanje_v_Ljubljani" title="Akademija za likovno umetnost in oblikovanje v Ljubljani">Akademija za likovno umetnost in oblikovanje</a></li>, <li><a href="/wiki/Biotehni%C5%A1ka_fakulteta_v_Ljubljani" title="Biotehniška fakulteta v Ljubljani">Biotehniška fakulteta</a></li>, <li><a href="/wiki/Ekonomska_fakulteta_v_Ljubljani" title="Ekonomska fakulteta v Ljubljani">Ekonomska fakulteta</a></li>, <li><a href="/wiki/Fakulteta_za_arhitekturo_v_Ljubljani" title="Fakulteta za arhitekturo v Ljubljani">Fakulteta za arhitekturo</a></li>, <li><a href="/wiki/Fakulteta_za_dru%C5%BEbene_vede_v_Ljubljani"

Dobimo torej seznam objektov razreda `element.Tag`, kjer posamezen element ustreza eni postavki v naštevnem seznamu, torej ustreza nazivu ene od fakultet Univerze v Ljubljani. V tem trenutku smo pripravljeni se znebiti oznak HTML in ohraniti le besedila posameznih elementov HTML, kar nam omogoča atribut `text`:

In [7]:
fakultete = [f.text for f in fakultete_html]
print(fakultete)

['Akademija za glasbo', 'Akademija za gledališče, radio, film in televizijo', 'Akademija za likovno umetnost in oblikovanje', 'Biotehniška fakulteta', 'Ekonomska fakulteta', 'Fakulteta za arhitekturo', 'Fakulteta za družbene vede', 'Fakulteta za elektrotehniko', 'Fakulteta za farmacijo', 'Fakulteta za gradbeništvo in geodezijo', 'Fakulteta za kemijo in kemijsko tehnologijo', 'Fakulteta za matematiko in fiziko', 'Fakulteta za pomorstvo in promet', 'Fakulteta za računalništvo in informatiko', 'Fakulteta za socialno delo', 'Fakulteta za strojništvo', 'Fakulteta za šport', 'Fakulteta za upravo', 'Filozofska fakulteta', 'Medicinska fakulteta', 'Naravoslovno-tehniška fakulteta', 'Pedagoška fakulteta', 'Pravna fakulteta', 'Teološka fakulteta', 'Veterinarska fakulteta', 'Zdravstvena fakulteta']


Dobili smo želeni seznam nazivov fakultet članic univerze v Ljubljani. Na koncu ga pretvorimo v tip zaporedje:

In [8]:
import pandas as pd

fakultete = pd.Series(fakultete)
print(fakultete)

0                                   Akademija za glasbo
1     Akademija za gledališče, radio, film in televi...
2          Akademija za likovno umetnost in oblikovanje
3                                 Biotehniška fakulteta
4                                   Ekonomska fakulteta
5                              Fakulteta za arhitekturo
6                            Fakulteta za družbene vede
7                           Fakulteta za elektrotehniko
8                                Fakulteta za farmacijo
9                Fakulteta za gradbeništvo in geodezijo
10          Fakulteta za kemijo in kemijsko tehnologijo
11                    Fakulteta za matematiko in fiziko
12                     Fakulteta za pomorstvo in promet
13            Fakulteta za računalništvo in informatiko
14                           Fakulteta za socialno delo
15                             Fakulteta za strojništvo
16                                   Fakulteta za šport
17                                  Fakulteta za

V nadaljevanju si bomo želeli iz nazivov fakultet pridelati njihove kratice (npr. kratica za našo fakulteto je FMF). Zato bomo potrebovali regularne izraze, ki jih bomo spoznali v drugem delu predavanja.

## Uvoz tabelaričnih podatkov s spleta

Preden nadaljujemo z regularnimi izrazi, ki nam omogočajo nadaljnjo obdelavo in čiščenje podatkov uvoženih s spleta, spoznajmo še en način uvoza spletnih podatkov. Ta je sicer bolj omejen, ker omogoča zgolj uvoz podatkov, zapisanih v tabelarični obliki. Je pa veliko bolj enostaven, saj zadostuje klic ene same funkcije knjižnice `pandas`, [`read_html`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html):

In [9]:
tabele = pd.read_html("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)")
print(len(tabele))

7


Rezultat funkcije je seznam podatkovnih tabel, s podatki iz vseh tabel na spletni strani s podanim naslovom. Za nas zanimiva je npr. tretja tabela v dobljenem seznamu, ki podaja nominalni bruto domači proizvod za posamezne države:

In [10]:
tabela = tabele[2]
print(tabela)

    Country/Territory UN region IMF[1][13]            World Bank[14]  \
    Country/Territory UN region   Forecast       Year       Estimate   
0               World         —  104476432       2023      100562011   
1       United States  Americas   26949643       2023       25462700   
2               China      Asia   17700899  [n 1]2023       17963171   
3             Germany    Europe    4429838       2023        4072192   
4               Japan      Asia    4230862       2023        4231141   
..                ...       ...        ...        ...            ...   
209             Palau   Oceania        267       2023              —   
210          Kiribati   Oceania        246       2023            223   
211             Nauru   Oceania        150       2023            151   
212        Montserrat  Americas          —          —              —   
213            Tuvalu   Oceania         63       2023             60   

               United Nations[15]             
          Year  

To je na prvi pogled lepo urejena podatkovna tabela, a vidimo, da se npr. v stolpcu leto pojavijo tudi vrednosti `[n 3]2022`, ker funkcija `read_html` uvozi tudi različne oznake, ki so jih ustvarjalci spletne strani (in tabele) dodali podatkom. Zato, da se znebimo teh oznak, moramo podatke **prečistiti**, ker nam spet omogočajo že večkrat omenjeni regularni izrazi.

## Regularni izrazi

Regularni izraz (angl. _regular expression_) je poseben niz znakov, ki ga uporabljamo za kratek zapis cele množice običajnih nizov znakov. Za podan regularni izraz $r$, z $L(r)$ označimo množico nizov znakov, ki jih regularni izraz zapiše. Množico $L(r)$ imenujemo _jezik regularnega izraza_. Rečemo tudi, da regularni izraz $r$ _tvori_ nize znakov iz jezika $L(r)$. Za vsak niz znakov iz $L(r)$ rečemo, da se _ujema z regularnim izrazom_ ali pa _ustreza regularnemu izrazu_ $r$.

Regularni izraz lahko tvorimo s pomočjo treh operatorjev.

1. Operator **stika** $\cdot$. Za množico nizov znakov $L(r_1 \cdot r_2)$, ki jih tvori stik dveh regularnih izrazov $r_1$ in $r_2$ velja

    $$ L(r_1 \cdot r_2) = \{ s_1 \cdot s_2 : s_1 \in L(r_1) \land s_2 \in L(r_2) \}, $$

    kjer je $s_1 \cdot s_2$ niz znakov, ki ga dobimo tako, da zapišemo oba niza znakov $s_1$ in $s_2$ en za drugim. Jezik, ki ga definira stik regularnih izrazov $\text{abc} \cdot \text{def}$, vsebuje zgolj en niz znakov $\text{abcdef}$.
    V regularnih izrazih pogosto operator stika ne pišemo, podobno kot v zapisu matematičnih izrazih ne pišemo operatorja množenja.

1. Operator **unije** (izbire) $|$. Za množico nizov znakov $L(r_1 | r_2)$, ki jih tvori unija dveh regularnih izrazov $r_1$ in $r_2$ velja

    $$ L(r_1 | r_2) = \{ s : s \in L(r_1) \lor s \in L(r_2) \} = L(r_1) \cup L(r_2). $$

    Z drugimi besedami, v jeziku unije dveh regularnih izrazov so nizi znakov, ki pripadajo jeziku vsaj enega izmed njih. Jezik regularnega izraza $\text{abc}|\text{def}$ je tako množica nizov znakov $\{ \text{abc}, \text{def} \}$.

1. Operator **iteracije** (ponavljanja) $^*$. Za množico nizov znakov $L(r^*)$, ki jih tvori iteracija regularnega izraza $r$ velja

    $$ L(r^*) = \{ s_1 \cdot s_2 \cdot \ldots \cdot s_n : n \geq 0 \land s_i \in L(r) \}. $$
    
    V jeziku iteracije regularnega izraza so torej nizi znakov, ki jih lahko zapišemo kot stik poljubnega števila nizov znakov iz jezika $L(r)$. Poljubno število je tudi 0, kar pomeni da je v jeziku $L(r)$ vedno prazen niz znakov, ki ga označimo z $\epsilon$. Tako je, na primer, jezik regularnega izraza $(\text{abc})^*$ neskončna množica nizov znakov $\{ \epsilon, \text{abc}, \text{abcabc}, \text{abcabcabc}, \ldots \}$.

Najvišjo prioriteta ima operator iteracije, temu sledi stik in na koncu operator unije. Na primer, $L(\text{ab}|\text{c}^*) = \{ ab, \epsilon, c, cc, ccc, \ldots \}$, po drugi strani pa $L(\text{a}(\text{b}|\text{c})^*) = \{ a, ab, ac, abb, abc, acb, acc, \ldots \} $.

Zanimive množice nizov znakov dobimo takrat, ko začnemo kombinirati te tri operatorje. Oglejmo si nekaj primerov regularnih izrazov za jezike nizov znakov sestavljenih iz znakov 0 in 1.

* $(0|1)^*$: poljuben niz znakov 0 in 1;
* $101(0|1)^*$: poljuben niz znakov 0 in 1 s predpono 101;
* $(0|1)^*(010|101)(0|1)^*$: poljuben niz znakov 0 in 1, ki vsebuje enega od nizov 010 in 101;
* $(00)^*$: poljuben niz sodega števila znakov 0;
* $(00|01|10|11)^*$: poljuben niz sodega števila znakov 0 ali 1;
* $(0|1)(00|01|10|11)^*$: poljuben niz lihega števila znakov 0 ali 1.

### Pythonovski regularni izrazi

Regularne izraze v Pythonu pišemo med enojnimi ali dvojnimi narekovaji, tako kot običajne nize znakov, a z dodatno predpono `r` (pomen slednje bomo pojasnili pozneje). Operatorja _stik_ sploh ne pišemo (vsak simbol zapisan v regularnem izrazu je v stiku z naslednjim), za operatorja _unije_ in _iteracije_ pa uporabljamo znaka `|` in `*`. Python, poleg omenjenih treh operatorjev, ponuja še vrsto drugih simbolov, ki nam olajšajo sestavljanje regularnih izrazov. Razdelimo jih lahko v tri kategorije opisane v nadaljevanju.

Prva kategorija so posebni regularni izrazi, ki nam omogočajo kratek zapis **množice znakov**.

* `.` je regularni izraz, ki se ujema s poljubnim znakom, razen z znakom za novo vrstico `\n`. Če hočemo zapisati regularni izraz, ki se ujema zgolj z znakom za piko `.`, moramo znaku `.` dodati predpono `\`: zapišemo ga torej `\.`.

* `[...]` se ujema s poljubnim znakom iz `...`. Posebna varianta tega regularnega izraza je `[^...]`, ki se ujema s poljubnim znakom, ki ga ni v `...`. Seznam lahko krajšamo z uporabo znaka `-`, ki pomeni od-do. Nekaj primerov:

  * `[0123456789]` in `[0-9]` se ujemata s poljubno števko;
  * `[a-z]` in `[A-Z]` se ujemata s poljubno malo in veliko črko;
  * `[0-46-9]` se ujema s poljubno števko, razen s števko 5;
  * `[^0-9]` se ujema s poljubnim znakom, ki ni števka;
  * `[aeiouAEIOU]` se ujema s poljubnim samoglasnikom;
  * `[^aeiouAEIOU]` se ujema s poljubnim soglasnikom.<br><br>

* `\d` se ujema s poljubno desetiško števko (števko od 0 do 9), ekvivalenten `[0-9]`.

* `\D` se ujema s poljubnim znakom, ki **ni** števka, ekvivalenten `[^0-9]`.

* `\s` se ujema s poljubnim _praznim_ znakom, ekvivalenten `[ \t\n\r\f\v]`.

* `\S` se ujema s poljubnim znakom, ki **ni** _prazen_, ekvivalenten `[^ \t\n\r\f\v]`.

* `\w` se ujema s poljubnim znakom, ki je števka, črka ali `_`, ekvivalenten `[a-zA-Z0-9_]`.

* `\W` se ujema s poljubnim znakom, ki **ni** števka, črka ali `_`, ekvivalenten `[^a-zA-Z0-9_]`.

Druga kategorija so simboli za posebne vrste iteracij, ki jih imenujemo **kvantifikatorji** in omogočajo **omejevanje števila ponovitev v iteraciji**.

| Kvantifikator | Število ponovitev $n$ |
|:---|:---|
| `*` | $n \geq 0$ |
| `+` | $n \geq 1$ |
| `?` | $n \in \{0, 1\}$ |
| `{x}` | $n = x$ |
| `{x,}` | $n \geq x$ |
| `{,y}` | $0 \leq n \leq y$ |
| `{x,y}` | $x \leq n \leq y$ |

Tretja kategorija so **različni uporabni simboli**.

* `^` se ujema z začetkom niza znakov.

* `$` se ujema s koncem niza znakov.

## Zajem (angl. _capture_) (pod)niza znakov

Z regularnimi izrazi lahko tudi _zajamemo_ podniz znakov, ki se ujema z delom regularnega izraza. Del regularnega izraza za zajem zamejimo z navadnimi oklepaji `(` in `)`. Če je `r` regularni izraz z $n$ deli za zajem, potem v nizu znakov `s`, ki se ujema z `r`, je zajetih $n$ podnizov znakov, katerih vrednosti prikličemo z `\1`, `\2`, ... in `\n`.

Zajeto skupino znakov lahko uporabimo pri definiciji regularnega izraza zato, da se sklicujemo na del niza znakov, ki se ujema z delom za zajem. V nadaljevanju podajamo nekaj primerov. Pravo moč zajetih nizov znakov bomo videli potem, ko bomo spoznali funkcijo `str_replace` v naslednjem razdelku.

* `([:alpha:])[:alpha:]*\1` se ujema s katerokoli nizom črk, kjer je prva črka enaka zadnji. Zajeti niz znakov se sestoji iz prve črke, ki je zamejena z oklepaji, nato sledi poljubno število črk in na koncu se mora pojaviti zajeti niz znakov. Ta je enak prvi črki, torej zadnja črka mora biti enaka prvi.

* `(([:alpha:]+) (\d{4}))` v nizu znakov `maja 2014` zajame tri skupine `\1 = maja 2014`, `\2 = maja` in `\3 = 2014`. Pravzaprav v poljubni kombinaciji _meseca_ in _leta_ zajame tri skupine, _mesec_ in _leto_, _mesec_ in _leto_.

* `(\d+)x(\d+)` se ujema z nizi znakov, ki zapišejo dimenzije matrike, na primer, `2x3` ali `302x11`. Pri tem je prvi niz zajetih znakov `\1` je enak prvi komponenti dimenzije, torej `2` oziroma `302` v omenjenih primerih, drugo zajetje `\2` pa drugi komponenti, `3` oziroma `11`.

### Funkcije za delo z regularnimi izrazi iz knjižnice `re`

Delo z regularnimi izrazi v Pythonu nam omogoča standardni Pythonovski modul [`re`](https://docs.python.org/3/howto/regex.html). V nadaljevanju bomo spoznali nekaj osnovnih funkcij te knjižnice.

Regularni izraz ustvarimo z uporabo funkcije `compile`:

In [11]:
import re

beseda = "beseda"
beseda_s_presledki = " beseda "
besedilo = "Prvi stavek je kratek. Drugi stavek omenja tudi številke, na primer, 10 in 20."

ri1 = re.compile(r"\w+")
print(ri1)

re.compile('\\w+')


Predpono `r` pred nizom znakov `\w+`, ki definira regularni izraz nam omogoča bolj enostavno pisanje regularnih izrazov. Poskrbi zato, da se znak `\` ne upošteva kot ubežni znak (angl. `escape character`), ki ga v Python-u uporabljamo za posebne znake (kot je, na primer, znak za novo vrstico `\n`) ali pa pisanje narekovajev znotraj narekovajev (npr. `'\''`). S predpono `r` se izognemo taki interpretaciji: zapis `\n` pomeni dobesedno `\n` (in ne znak za novo vrstico, kot bi pomenil v običajnem Python-ovskem zapisu nizov znakov).

Funkcija `match`, ki je metoda razreda regularnih izrazov, preveri ujemanje regularnega izraza s podnizom na začetku podanega niza znakov in vrne zadetek ali `None`:

In [12]:
u1 = ri1.match(beseda)
print(f"Objekt ujemenja: {u1}")
print(f".span: {u1.span()}")
print(f".group: {u1.group()}")

u2 = ri1.match(beseda_s_presledki)
print(u2)

Objekt ujemenja: <re.Match object; span=(0, 6), match='beseda'>
.span: (0, 6)
.group: beseda
None


Rezultat je torej `None` (v primeru neujemanja) oziroma objekt razreda `Match object` z dvema metodama `span` in `group`. Funkcija `search` je podobna `match`, le da preverja ujemanje regularnega izraza s poljubim podnizom podanega niza znakov in vrne prvi zadetek ali `None`:

In [13]:
u3 = ri1.search(beseda_s_presledki)
print(u3)

<re.Match object; span=(1, 7), match='beseda'>


Spoznajmo na koncu še funkcijo `findall`: za podan niz znakov vrne vse njegove podnize, ki se ujemajo s podanim regularnim izrazom:

In [14]:
print(ri1.findall(beseda))
print(ri1.findall(beseda_s_presledki))
print(ri1.findall(besedilo))

['beseda']
['beseda']
['Prvi', 'stavek', 'je', 'kratek', 'Drugi', 'stavek', 'omenja', 'tudi', 'številke', 'na', 'primer', '10', 'in', '20']


Vidimo torej, da naš regularni izraz `ri1` lahko uporabimo za hitro razčlenjevanje dolgih nizov znakov (besedil) na posamezne besede. Kar je lahko sila uporabno pri obdelavi in čiščenju podatkov pridobljenih iz spletnih strani.

V nadaljnjem premisleku, bi lahko tak regularni izraz uporabili za pridelavo kratice iz podanega niza znakov. Definirajmo kratico najprej na primeru: kratica za besedno zvezo "regularni izraz" je "RI". Torej kratica je niz znakov sestavljen iz začetnih črk besed v podanem besedilu. Pri tem je običajno, da pri kraticah ne upoštevamo začetnic veznikov (veznih besed), pa še kratico sestavimo iz velikih črk:

In [15]:
def kratica(bz, velike = True, ri = re.compile(r"\w+")):
    vezniki = ["za", "in"]
    k = "".join([b[0] for b in ri.findall(bz) if b not in vezniki])
    return k.upper() if velike else k

print(kratica("regularni izraz"))
print(kratica("besedna zveza za test"))
print(kratica("Fakulteta za matematiko in fiziko"))

RI
BZT
FMF


Definirali smo torej funkcijo `kratica`, ki za podano besedilo vrne njegovo kratico. Funkcijo zdaj lahko uporabimo za sestavljanje tabele, v katerem bo prvi stolpec naziv fakultete, drugi pa kratica.

In [16]:
fakultete = pd.DataFrame({
    "ime": fakultete,
    "kratica": fakultete.apply(kratica)
})
print(fakultete)

                                                  ime kratica
0                                 Akademija za glasbo      AG
1   Akademija za gledališče, radio, film in televi...   AGRFT
2        Akademija za likovno umetnost in oblikovanje    ALUO
3                               Biotehniška fakulteta      BF
4                                 Ekonomska fakulteta      EF
5                            Fakulteta za arhitekturo      FA
6                          Fakulteta za družbene vede     FDV
7                         Fakulteta za elektrotehniko      FE
8                              Fakulteta za farmacijo      FF
9              Fakulteta za gradbeništvo in geodezijo     FGG
10        Fakulteta za kemijo in kemijsko tehnologijo    FKKT
11                  Fakulteta za matematiko in fiziko     FMF
12                   Fakulteta za pomorstvo in promet     FPP
13          Fakulteta za računalništvo in informatiko     FRI
14                         Fakulteta za socialno delo     FSD
15      

Zaradi dvoumnih kratic v nekaterih primerih, na Univerzi v Ljubljani so sprejeli še te tri izjeme za kratice, k jih v tabelo moramo vnesti na roke:

In [17]:
fakultete.loc[fakultete.ime == "Fakulteta za farmacijo", "kratica"] = "FFA"
fakultete.loc[fakultete.ime == "Fakulteta za šport", "kratica"] = "FSP"
fakultete.loc[fakultete.ime == "Pedagoška fakulteta", "kratica"] = "PEF"
print(fakultete)

                                                  ime kratica
0                                 Akademija za glasbo      AG
1   Akademija za gledališče, radio, film in televi...   AGRFT
2        Akademija za likovno umetnost in oblikovanje    ALUO
3                               Biotehniška fakulteta      BF
4                                 Ekonomska fakulteta      EF
5                            Fakulteta za arhitekturo      FA
6                          Fakulteta za družbene vede     FDV
7                         Fakulteta za elektrotehniko      FE
8                              Fakulteta za farmacijo     FFA
9              Fakulteta za gradbeništvo in geodezijo     FGG
10        Fakulteta za kemijo in kemijsko tehnologijo    FKKT
11                  Fakulteta za matematiko in fiziko     FMF
12                   Fakulteta za pomorstvo in promet     FPP
13          Fakulteta za računalništvo in informatiko     FRI
14                         Fakulteta za socialno delo     FSD
15      

Sestavljanje te tabele ilustrira težave, ki nastajajo pri uvozu podatkov s spletnih strani, od samega uvoza v Pyhon-ovske in podatkovne strukture `pandas`, do popravljanja in čiščenja uvoženih podatkov.

# Naloge

1. Iz datoteke [`enaslovi.txt`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/enaslovi.txt) preberi naslove e-pošte osmih (namišljenih) sodelavcev Univerze v Ljubljani. Uvozi te podatke in iz njih, v `pandas` sestavi podatkovno tabelo s štirimi stolpci (spremenljivkami): ime, priimek, fakulteta in enaslov.

    Pri tem upoštevaj, da naslovi e-pošte na Univerzi v Ljubljani so sestavljeni po naslednjem vzorcu `ime.priimek@kratica.uni-lj.si`, pri čemer je `ime` in `priimek` očitno ustrezata imenu in priimku sodelavke, `kratica` pa kratici fakultete, kjer je zaposlena. Lahko si pomagaš s tabelo nazivov in kratic fakultet, ki smo jo sestavili med predavanji.

1. V Python-u definiraj funkcijo `n_samoglasnikov`, ki za podan niz znakov vrne število samoglasnikov v njemu. Nato definiraj še funkcijo `n_soglasnikov`, ki prešteje število soglasnikov v podanem nizu znakov.

1. V datoteki [`besedilo.txt`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/besedilo.txt) je izbrano besedilo iz slovenske različice enciklopedije Wikipedia. S pomočjo funkcij iz prejšnje naloge, v `pandas` sestavi tabelo `besede` s tremi spremenljivkami: dimenzijsko spremenljivko `beseda`, katere domena je množica vseh različnih besed iz besedila v datoteki `besedilo.txt`, in merjeni spremenljivki `n_samoglasnikov` in `n_soglasnikov`, ki podajata število samoglasnikov in soglasnikov v izbrani besedi.

1. V članku [Statistične regije Slovenije](https://sl.wikipedia.org/wiki/Statisti%C4%8Dne_regije_Slovenije) v slovenski različici spletne enciklopedije Wikipedia je poglavje _Občine po statističnih regijah_, ki podaja sezname občin za vsako izmed dvanajstih statističnih regij v Sloveniji.
    
    Napiši program v `pandas`, ki iz podatkov v tem članku sestavi urejeno podatkovno tabelo `obcina_regija` z dimenzijsko spremenljivko `obcina` in merjeno spremenljivko `regija`, ki določi regijo izbrane občine v Sloveniji.

1. Iz članka omenjenega v prejšnji nalogi uvozi v Python podatkovno tabelo (glej poglavje _Statistične regije_ v tem članku), ki za vsako izmed dvanajstih statističnih regij poda število prebiralcev in površino v kvadratnih kilometrih.